<a href="https://www.kaggle.com/code/averma111/autogluon-pss3e18?scriptVersionId=135189469" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%%capture
!pip install autogluon


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.2/372.2 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.2/116.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 17.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 30.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os


import torch
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
from autogluon.common import space
import os.path


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler


import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
import itertools

In [25]:
train_data = TabularDataset('/kaggle/input/playground-series-s3e18/train.csv')
subsample_size = 14838  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=42)
train_data.drop(columns=['EC3', 'EC4', 'EC5', 'EC6'],axis=1,inplace=True)
train_data.head()

Loaded data from: /kaggle/input/playground-series-s3e18/train.csv | Columns = 38 / 38 | Rows = 14838 -> 14838


,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2
11988,11988,118.000000,3.060660,2.536175,2.536175,1.640774,1.640774,0.548756,0.182919,17.721856,...,0.000000,0.000000,11.752550,12.841643,9.589074,33.833333,1,1,1,1
13039,13039,378.018438,10.685872,7.066210,7.066210,6.041834,6.041834,4.039540,2.503411,12.514062,...,0.000000,0.000000,23.468091,24.539800,13.825658,44.000000,0,0,0,1
13637,13637,277.172776,1.732051,0.682574,1.462072,0.886443,0.886443,0.087310,0.000000,24.415866,...,0.000000,0.000000,11.938611,19.262465,9.589074,44.333333,2,2,1,1
5537,5537,377.203227,11.265362,9.264697,9.264697,6.445092,6.445092,4.308487,2.706995,42.723899,...,0.000000,0.000000,17.744066,45.448667,28.250470,64.083333,0,0,0,0
8425,8425,315.695337,5.985416,3.473678,3.473678,2.634453,2.634453,1.507705,0.672861,0.000000,...,17.696186,6.066367,0.000000,6.103966,0.000000,37.166667,0,0,1,1


In [26]:
class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str]
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str]
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        self.path = '/kaggle/working/'
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = self.eval_metrics[label]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = '/kaggle/working/'
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [33]:
labels = ['EC1','EC2']  # which columns to predict based on the others
problem_types = ['multiclass','binary']  # type of each prediction problem
save_path = 'agModels-predictEC1EC2'  # specifies folder to store trained models
time_limit =600

In [34]:
multi_predictor = MultilabelPredictor(labels=labels, problem_types=problem_types, path=save_path)
multi_predictor.fit(train_data, time_limit=time_limit)

Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "/kaggle/working/Predictor_EC1/"
AutoGluon Version:  0.8.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Jun 24 10:55:41 UTC 2023
Disk Space Avail:   20.33 GB / 20.96 GB (97.0%)
Train Data Rows:    14838
Train Data Columns: 32
Label Column: EC1
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Train Data Class Count: 2
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    31583.64 MB
	Train Data (Original)  Memory Usage: 3.8 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting 

Fitting TabularPredictor for label: EC1 ...


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 28 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])   :  4 | ['id', 'NumHeteroatoms', 'fr_COO', 'fr_COO2']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 28 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])   :  4 | ['id', 'NumHeteroatoms', 'fr_COO', 'fr_COO2']
	0.2s = Fit runtime
	32 features in original data used to generate 32 features in processed data.
	Train Data (Processed) Memory Usage: 3.8 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.28s ...
AutoGluon will gauge predictive performance using evaluation metric: 'accuracy'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 13354, Val Rows: 1484
User-specified model hyperparameters to be fit:
{
	'NN_TORCH': {},
	'GBM': [{'extra_trees': True,

Fitting TabularPredictor for label: EC2 ...


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 28 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])   :  5 | ['id', 'NumHeteroatoms', 'fr_COO', 'fr_COO2', 'EC1']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 28 | ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', ...]
		('int', [])       :  4 | ['id', 'NumHeteroatoms', 'fr_COO', 'fr_COO2']
		('int', ['bool']) :  1 | ['EC1']
	0.3s = Fit runtime
	33 features in original data used to generate 33 features in processed data.
	Train Data (Processed) Memory Usage: 3.81 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.3s ...
AutoGluon will gauge predictive performance using evaluation metric: 'accuracy'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 13354, Val Rows: 1484
User-specified model hyperparameters to be fit:

MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('/kaggle/working/')


In [35]:
test_data = TabularDataset('/kaggle/input/playground-series-s3e18/test.csv')
test_data = test_data.sample(n=9893, random_state=42)
test_data.head()

Loaded data from: /kaggle/input/playground-series-s3e18/test.csv | Columns = 32 / 32 | Rows = 9893 -> 9893


,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2
2807,17645,327.399591,4.414719,2.618262,2.618262,1.670820,1.670820,1.070620,0.694224,17.282269,...,0.000000,0.000000,18.199101,6.544756,10.902925,0.000000,0.000000,23.166667,0,0
5235,20073,292.492090,4.663902,2.767073,2.767073,1.904314,1.904314,1.060948,0.361460,5.969305,...,5.969305,0.000000,0.000000,6.076020,5.969305,0.000000,0.000000,37.666667,1,1
6291,21129,1122.912455,16.188780,10.684028,13.674919,8.012199,11.008637,7.199415,3.408511,89.249895,...,21.335138,0.000000,13.847474,18.181117,15.645394,69.410022,31.961948,73.715826,0,0
9789,24627,1727.925403,24.910940,15.637880,20.567315,11.722242,13.672761,11.312963,4.827185,90.658938,...,23.468091,32.533097,0.000000,37.099000,69.141353,63.436851,50.697492,119.375115,0,0
2868,17706,62.568425,2.642734,1.049739,1.049739,0.504904,0.504904,0.142577,0.000000,5.969305,...,28.148794,0.000000,0.000000,0.000000,5.969305,6.544756,4.794537,21.178241,1,1


In [38]:
multi_predictor = MultilabelPredictor.load(save_path)  
multi_predictor.evaluate(train_data)

Evaluating TabularPredictor for label: EC1 ...


Evaluation: accuracy on test data: 0.8531473244372557
Evaluations on test data:
{
    "accuracy": 0.8531473244372557,
    "balanced_accuracy": 0.7854774472817261,
    "mcc": 0.6691886841310939
}


Evaluating TabularPredictor for label: EC2 ...


Evaluation: accuracy on test data: 0.9608437794851058
Evaluations on test data:
{
    "accuracy": 0.9608437794851058,
    "balanced_accuracy": 0.9039946594599823,
    "mcc": 0.875912748407303,
    "roc_auc": 0.9714196134343737,
    "f1": 0.9760599942313241,
    "precision": 0.9540840985983567,
    "recall": 0.9990721214677352
}


{'EC1': {'accuracy': 0.8531473244372557,
  'balanced_accuracy': 0.7854774472817261,
  'mcc': 0.6691886841310939},
 'EC2': {'accuracy': 0.9608437794851058,
  'balanced_accuracy': 0.9039946594599823,
  'mcc': 0.875912748407303,
  'roc_auc': 0.9714196134343737,
  'f1': 0.9760599942313241,
  'precision': 0.9540840985983567,
  'recall': 0.9990721214677352}}

In [42]:
predictions = multi_predictor.predict_proba(test_data)
print("Predictions:  \n", predictions)

Predicting with TabularPredictor for label: EC1 ...
Predicting with TabularPredictor for label: EC2 ...
Predictions:  
 {'EC1':              0         1
2807  0.241953  0.758047
5235  0.238493  0.761507
6291  0.365012  0.634988
9789  0.235141  0.764859
2868  0.196513  0.803487
...        ...       ...
5734  0.271537  0.728463
5191  0.238284  0.761716
5390  0.618955  0.381045
860   0.197483  0.802517
7270  0.505737  0.494263

[9893 rows x 2 columns], 'EC2':              0         1
2807  0.099513  0.900487
5235  0.075354  0.924646
6291  0.481706  0.518294
9789  0.258135  0.741865
2868  0.085378  0.914622
...        ...       ...
5734  0.154639  0.845361
5191  0.097134  0.902866
5390  0.022749  0.977251
860   0.187148  0.812852
7270  0.029129  0.970871

[9893 rows x 2 columns]}


In [56]:
EC1 = list(predictions.values())
EC2 = list(predictions.values())

In [57]:
class Submit:
    
    def submit_predictions(self,test_data):
        df_submit = pd.DataFrame(data={'id': test_data['id'],'EC1':EC1[0][1],'EC2':EC2[1][1]})
        df_submit.to_csv('submission.csv',index=False)
        print('Submission Completed!!')
        return df_submit
        
        
submit = Submit()
df_submit=submit.submit_predictions(test_data)

Submission Completed!!


In [58]:
df_submit

,id,EC1,EC2
2807,17645,0.758047,0.900487
5235,20073,0.761507,0.924646
6291,21129,0.634988,0.518294
9789,24627,0.764859,0.741865
2868,17706,0.803487,0.914622
...,...,...,...
5734,20572,0.728463,0.845361
5191,20029,0.761716,0.902866
5390,20228,0.381045,0.977251
860,15698,0.802517,0.812852
